## Create an AI engine
Create a Driverless AI engine for access to automated machine learning to build models for us on our data. 

See the `2 AI Engines` tutorial for more details on how to use and interact with **Steam** for creating and managing your AI Engines.


In [2]:
from h2o_ai_cloud import steam_client
from h2osteam.clients import DriverlessClient

## Securely connect 
We first connect to the H2O AI Cloud using our platform token to create a token provider object. We can then use this object to log into Steam and other APIs.

In [3]:
steam = steam_client()

Visit https://cloud-dev.h2o.ai/auth/get-platform-token to get your platform token


In [5]:
# Profiles determin the resourcs that an AI Engine will have access to - see the Steam tutorial for more information
dai_profiles = [profile["name"] for profile in steam.get_profiles() if profile["profile_type"] == "driverless_kubernetes"]

print(dai_profiles)

['jg-driverless-kubernetes', 'aaa', 'admin-driverless-kubernetes', 'dai-with-h2o-drive-k8s', 'default-driverless-kubernetes-low-mem', 'gbq-driverless-kubernetes', 'default-driverless-kubernetes', 'visitor-driverless-kubernetes', 'se-driverless-kubernetes', 'dai-multinode-tmpfs', 'dai-multinode-tmpfs-14cores', 'multinode-single-user', 'multinode-single-user-fixed-workers', 'multinode-single-user-no-dask', 'multinode-single-user-no-gpu', 'default-driverless-kubernetes-migration', 'multinode-single-user-cpu', 'default-driverless-kubernetes-max_cores8', 'custom-driverless-cpu-test', 'custom-driverless-cpu-test-4core']


In [12]:
# Get the migration profile
migration_profile = "default-driverless-kubernetes-migration"
dai_profiles = [profile for profile in dai_profiles if profile == migration_profile]
print(dai_profiles)

['default-driverless-kubernetes-migration']


In [26]:
# Create DAI instance
DAI_VERSION = "1.10.4.99"
dai_instance_name = "joby-dai-e2e-migration"

try:
    dai_instance = DriverlessClient(steam).get_instance(name=dai_instance_name)
except Exception as e:
    print(f'Starting new DAI {DAI_VERSION} instance')
    dai_instance = DriverlessClient(steam).launch_instance(
         name = dai_instance_name,
         version = DAI_VERSION,
         profile_name=dai_profiles[0],
     )
status = dai_instance.status()
print(f'DAI status is {status}')

if status == "stopped":
    dai_instance.start()

print(f'DAI run status is {dai_instance.status()}')

DAI status is running
DAI run status is running


In [28]:
# Get the DAI python client
dai_client = dai_instance.connect()

## Driverless python client APIs
DAI functionality with python client

In [32]:
print(dai_client.server.version)
print(dai_client.connectors.list())

1.10.4
['upload', 'file', 'hdfs', 's3', 'recipe_file', 'recipe_url', 'h2o_drive', 'feature_store']


In [39]:
# List of experiments available
list_of_experiments = dai_client.experiments.list()
experiment = [exp for exp in list_of_experiments if '1.10.2' in exp.name][0]
print(experiment)

1.10.2-walmart-catboost-byor (c74a42bc-aeb1-11ec-8f4b-0242ac120002)


### Fetch experiment datasets


In [48]:
assert experiment.is_running() is False
print(experiment.datasets)
dataset = experiment.datasets.get('train_dataset')
print(dataset.key)

{'train_dataset': <class 'Dataset'> ff759814-aeaf-11ec-b224-0242ac120002 walmart_train.csv, 'validation_dataset': None, 'test_dataset': None}
ff759814-aeaf-11ec-b224-0242ac120002


In [55]:
# Create a test project
project_name = "dai_e2e_test_migration"
projects = dai_client.projects.list()

list_of_project = [p for p in projects if project_name in p.name]
if len(list_of_project) == 0 :
    project = dai_client.projects.create(
        name = project_name,
        description = "Test migration",
    )
else:
    project = list_of_project[0]
print(project.name)

dai_e2e_test_migration


In [56]:
# Attach experiment and dataset to the project
project.link_experiment(experiment = experiment)

print(project.experiments)
print(project.datasets)

    | Type       | Key                                  | Name
----+------------+--------------------------------------+------------------------------
  0 | Experiment | c74a42bc-aeb1-11ec-8f4b-0242ac120002 | 1.10.2-walmart-catboost-byor
{'test_datasets':     | Type   | Key   | Name
----+--------+-------+--------, 'train_datasets':     | Type    | Key                                  | Name
----+---------+--------------------------------------+-------------------
  0 | Dataset | ff759814-aeaf-11ec-b224-0242ac120002 | walmart_train.csv, 'validation_datasets':     | Type   | Key   | Name
----+--------+-------+--------}


### Query for the experiment artifacts in mlops


In [58]:
from h2o_ai_cloud import mlops_client

In [60]:
mlops = mlops_client()

Visit https://cloud-dev.h2o.ai/auth/get-platform-token to get your platform token


In [63]:
mlops_projects = mlops.storage.project.list_projects(body={}).project
for p in mlops_projects:
    print(p.id, p.display_name)

27498d52-15c5-4a48-b2fa-f136bc90522c dai_e2e_test_migration
492b8ec5-5a48-4551-870c-edcc606fc5c3 dai_e2e_migration
cca74583-bf46-492b-b017-4070197f7ae3 test_project


In [ ]:
# Stop DAI instance
dai_instance.stop()